In [ ]:
%pip install transformers
%pip install torch
%pip install sentencepiece
%pip install datasets
%pip install scikit-learn

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)


  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
    --------------------------------------- 3.1/204.1 MB 18.4 MB/s eta 0:00:11
   - -------------------------------------- 7.1/204.1 MB 18.2 MB/s eta 0:00:11
   -- ------------------------------------- 11.3/204.1 MB 18.5 MB/s eta 0:00:11
   -- ------------------------------------- 14.9/204.1 MB 18.8 MB/s eta 0:00:11
   --- ------------------------------------ 18.6/204.1 MB 18.4 MB/s eta 0:00:11
   ---- ----------------------------------- 22.3/204.1 MB 18.5 MB/s eta 0:00:10
   ----- ---------------------------------- 25.7/204.1 MB 18.3 MB/s eta 0:00:10
   ----- ---------------------------------- 29.4/204.1 MB 18.3 MB/s eta 0:00:10
   ------ --------------------------------- 33.3/204.1 MB 18.4 MB/s eta 0:00:10
   ------- -------------------------------- 37.2/204.1 MB 18.3 MB/s eta 0:00:10
   -------- ------------------------------- 41.2/204.1 MB 18.3 MB/s 

  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl (991 kB)


  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-19.0.1-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow-19.0.1-cp312-cp312-win_amd64.whl (25.3 MB)
Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl (30 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [2]:
sample_data = [
    {"text": "धान के लिए कौन सी खाद सबसे अच्छी है?", "language": "hi", "entities": ["धान", "खाद"]},
    {"text": "tomato ke liye mausam kaisa hai?", "language": "hi-en", "entities": ["tomato", "mausam"]},
    {"text": "Groundnut yield is low due to poor rainfall", "language": "en", "entities": ["Groundnut", "rainfall"]},
]


In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Multilingual BERT model
model_checkpoint = "Davlan/xlm-roberta-base-ner-hrl"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)


c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Device set to use cpu
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [4]:
for entry in sample_data:
    print(f"\nText: {entry['text']}")
    ner_results = ner_pipeline(entry['text'])
    for ent in ner_results:
        print(f" - Entity: {ent['word']}, Label: {ent['entity_group']}")



Text: धान के लिए कौन सी खाद सबसे अच्छी है?

Text: tomato ke liye mausam kaisa hai?

Text: Groundnut yield is low due to poor rainfall


In [5]:
def wsd_stub(text):
    if "crop" in text.lower() or "धान" in text:
        return "Crop Information"
    elif "weather" in text.lower() or "मौसम" in text:
        return "Weather Info"
    else:
        return "General Query"

for entry in sample_data:
    print(f"\nText: {entry['text']}")
    category = wsd_stub(entry['text'])
    print(f" - Intent (WSD Result): {category}")



Text: धान के लिए कौन सी खाद सबसे अच्छी है?
 - Intent (WSD Result): Crop Information

Text: tomato ke liye mausam kaisa hai?
 - Intent (WSD Result): General Query

Text: Groundnut yield is low due to poor rainfall
 - Intent (WSD Result): General Query


In [6]:
def chatbot_response(user_input):
    ner_entities = ner_pipeline(user_input)
    intent = wsd_stub(user_input)
    
    response = f"Detected intent: {intent}\nEntities: {[ent['word'] for ent in ner_entities]}"
    
    # Example response logic
    if intent == "Crop Information":
        response += "\n✅ Suggestion: Use organic fertilizer for better crop yield."
    elif intent == "Weather Info":
        response += "\n☁️ Current weather: Sunny, 32°C"
    else:
        response += "\n📌 Please provide more details."
    
    return response

# Try it out
chatbot_response("धान के लिए खाद कौन सी डालनी चाहिए?")


'Detected intent: Crop Information\nEntities: []\n✅ Suggestion: Use organic fertilizer for better crop yield.'